In [29]:
import numpy as np
from collections import defaultdict

In [30]:
# Parse the data: ONLY NEED TO WORK ON THIS: REST IS DONE!!
def parseData(filePath):
    data = []
    
    with open(filePath, 'r') as file:
        sentId = ""
        
        for line in file:
            line = line.strip()
            if line.startswith("# sent_id"):
                newLine = line.split("=")
                newSentence = newLine[-1].strip()
                sentId = newSentence
                
            elif line and not line.startswith("#"):
                divisions = line.split()
                if len(divisions) >= 5:
                    data.append([sentId, divisions[0], divisions[1], divisions[3]]) 
                    
    return data


data = parseData("./NLP2/train.txt")
print(data[:10])

[['GUM_academic_art-1', '1', 'Aesthetic', 'JJ'], ['GUM_academic_art-1', '2', 'Appreciation', 'NN'], ['GUM_academic_art-1', '3', 'and', 'CC'], ['GUM_academic_art-1', '4', 'Spanish', 'JJ'], ['GUM_academic_art-1', '5', 'Art', 'NN'], ['GUM_academic_art-1', '6', ':', ':'], ['GUM_academic_art-2', '1', 'Insights', 'NNS'], ['GUM_academic_art-2', '2', 'from', 'IN'], ['GUM_academic_art-2', '3', 'Eye', 'NN'], ['GUM_academic_art-2', '4', '-', 'HYPH']]


In [31]:
# Perform the actions of LEFT ARC, RIGHT ARC, SHIFT and REDUCE
def performAction(configuration, action):
    stack, buffer, arcs = configuration   
    topS = stack[-1] if stack else None
    firstB = buffer[0] if buffer else None
    
    if action == "LEFT-ARC":
        arcs.append((firstB, topS))
        stack.pop()
    elif action == "RIGHT-ARC":
        arcs.append((topS, firstB))
        stack.append(firstB)
        buffer.pop(0)
    elif action == "REDUCE":
        stack.pop()
    elif action == "SHIFT":
        buffer.pop(0)
        stack.append(firstB)
        
    return (stack, buffer, arcs)

In [32]:
def oracle(configuration, DT):
    stack, buffer, arcs = configuration
    
    topS = stack[-1] if stack else None
    firstB = buffer[0] if buffer else None

    if firstB and topS and (firstB, topS) in DT and topS not in [D for H, D in arcs]:
        return "LEFT-ARC"
    elif topS and firstB and (topS, firstB) in DT:
        return "RIGHT-ARC"
    elif topS and firstB and topS in [D for H, D in arcs]:
        for word in reversed(stack[:-1]):
            if (word, firstB) in DT or (firstB, word) in DT:
                return "REDUCE"
    return "SHIFT"

stack = []
buffer = ['He', 'sent', 'her', 'a', 'letter', '.']
arcs = []
dependency_tree = [('sent', 'He'), ('sent', 'her'), ('letter', 'a'), ('sent', 'letter'), ('sent', '.')]

while buffer:
    action = oracle((stack, buffer, arcs), dependency_tree)
    print(stack, buffer, arcs, action)
    T.append(((stack, buffer, arcs),action))
    stack, buffer, arcs = performAction((stack, buffer, arcs), action)


[] ['He', 'sent', 'her', 'a', 'letter', '.'] [] SHIFT
['He'] ['sent', 'her', 'a', 'letter', '.'] [] LEFT-ARC
[] ['sent', 'her', 'a', 'letter', '.'] [('sent', 'He')] SHIFT
['sent'] ['her', 'a', 'letter', '.'] [('sent', 'He')] RIGHT-ARC
['sent', 'her'] ['a', 'letter', '.'] [('sent', 'He'), ('sent', 'her')] SHIFT
['sent', 'her', 'a'] ['letter', '.'] [('sent', 'He'), ('sent', 'her')] LEFT-ARC
['sent', 'her'] ['letter', '.'] [('sent', 'He'), ('sent', 'her'), ('letter', 'a')] REDUCE
['sent'] ['letter', '.'] [('sent', 'He'), ('sent', 'her'), ('letter', 'a')] RIGHT-ARC
['sent', 'letter'] ['.'] [('sent', 'He'), ('sent', 'her'), ('letter', 'a'), ('sent', 'letter')] REDUCE
['sent'] ['.'] [('sent', 'He'), ('sent', 'her'), ('letter', 'a'), ('sent', 'letter')] RIGHT-ARC


In [6]:
def getBOW (word, getIndex):
    if word is None:
        return [0] * len(getIndex)
    else:
        index = getIndex[word]
        return [1 if i == index else 0 for i in range(len(getIndex))]

In [3]:
def getFeatureModel(configuration, index, pos, depT):
    stack, buffer, arcs = configuration
    heads = defaultdict(str)
    
    # pos is 2d array of dictionary from string to string i.e. [dict{string : string}]
    # dep is 2d array of dictionary from string to string which gives us the modifier for a non head word
    topS = stack[-1] if stack else None
    firstB = buffer[0] if buffer else None
    posTagTopS = pos[index][topS] if topS else None
    posTagFirstB = pos[index][topS] if firstB else None
    posTagSecondB = pos[index][buffer[1]] if len(buffer) > 1 else None
    
    topDep = None
    topLDep = None
    topRDep = None
    firstLDep = None
    
    for head, depTag in arcs:
        if topS and head == topS:
            topDep = depTag
            if not topLDep:
                topLDep = depTag
            topRDep = depTag
        
        if firstB and head == firstB:
            if not firstLDep:
                firstLDep = depTag
                
    topDep = depT[index][topDep] if topDep and depT[index][topDep] else None
    topLDep = depT[index][topLDep] if topLDep and depT[index][topLDep] else None
    topRDep = depT[index][topRDep] if topRDep and depT[index][topRDep] else None
    firstLDep = depT[index][firstLDep] if firstLDep and depT[index][firstLDep] else None
    
    
    # We need to find the bag of words value of the conditions
    # words : dictionary of indices of all unique words to be used in the problem
    # posTags : dictionary of indices of all unique pos tages to be used in this problem
    # depTags : dictionary of indices of all unique posTags to be used in this problem
    
    tops = BOW(topS, words)
    first = BOW(firstB, words)
    posTop = BOW(posTagTopS, posTags)
    posFirst = BOW(posTagFirstB, posTags)
    posSecond = BOW(posTagSecondB, posTags)
    topDeps = BOW(topDep, depTags)
    topLDeps = BOW(topLDep, depTags)
    topRDeps = BOW(topRDep, depTags)
    firstLDeps = BOW(firstLDep, depTags)
    
    fM =  tops + first + posTop + posFirst + posSecond + topDeps + topLDeps + topRDeps + firstLDeps
    fM = fM + fM + fM + fM 
    
    return fM

In [6]:
transitionValue = {
    "LEFT-ARC" : 0, "RIGHT-ARC" : 1, "REDUCE" : 2, "SHIFT" : 3
}

getAction = ["LEFT-ARC", "RIGHT-ARC", "REDUCE", "SHIFT"]

In [12]:
def performComputation(fM, v, action):
    result = [0] * len(fM)
    
    if action == 'LEFT-ARC':
        result[:v] = [1] * v
    elif action == 'RIGHT-ARC':
        result[v:2*v] = [1] * v
    elif action == 'REDUCE':
        result[2*v:3*v] = [1] * v
    elif action == 'SHIFT':
        result[3*v:] = [1] * v
        
    return [fm * res for fm, res in zip(fM, result)]
        

def dotProduct(weights, features):
    sumValue = 0.0
    for i in range(min(len(weights), len(features))):
        sumValue += weights[i]*features[i]

In [13]:
# Online Learning Algorithm
# DT should be a 2D array of tupes head -> dep for each token
# listOfConfigs is a 2D vector of tokenized words in buffer, arcs and stack being empty
def learnTheWeights(listOfConfigurations, maxIters, DT):
    v = (2*len(words) + 3*len(posTags) + 4*len(depTags))
    w = [1.1] * 4 * v
    for i in range(maxIters):
        for j in range(len(listOfConfigurations)):
            c = listOfConfigurations[j]
            stack, buffer, arc = c
            maxValue = -inf
            trainingAction = "LEFT-ARC"
            features = []
            while buffer:
                fM = getFeatureModel(c, j)
                for action in getAction:
                    newFeatureVector = performComputation(fM, v, action)
                    value = dotProduct(newFeatureVector, w)
                    
                    if value > maxValue:
                        value = maxValue
                        features = newFeatureVector
                        trainingAction = action
                
                oracleAction = oracle(c, DT[j])
                
                if trainingAction != oracleAction:
                    w = w + performComputation(fM, v, oracleAction) - performComputation(fM, v, trainingAction)
                stack, buffer, arc = performAction(c, oracleAction)
        
        return w        

In [11]:
# Online Learning Algorithm:
Result = []
# testData is the list of Initial Configs for all test data
# Note: we need to gather the features over both the training as well as the test examples
def defineDependencyGraph(testData):
    for i in range(len(testData)):
        c = testData[i]
        stack, buffer, arcs = c
        maxValue = -inf
        trainingAction = "LEFT-ARC"
        features = []
        while buffer:
            fM = getFeatureModel(c, i+2000)
            for action in getAction:
                newFeatureVector = performComputation(fM, v, action)
                value = dotProduct(newFeatureVector, w)
                    
                if value > maxValue:
                    value = maxValue
                    features = newFeatureVector
                    trainingAction = action
            stack, buffer, arc = performAction(c, trainingAction)
            
        Result.append(arcs)

[0, 0, 0, 0, 5, 6, 0, 0]
